<td>
   <a target="_blank" href="https://beewant.com" ><img src="https://media.licdn.com/dms/image/C4E0BAQFKjwZohF7zRA/company-logo_200_200/0/1655303818792?e=2147483647&v=beta&t=fmLrigj1HhqskB4jNk6aDb6XASTUx1SnRvMXWBPtuCY"></a>
</td>
<p><B>Welcome to Beewant Platform Tutorial</B></p>

<td>
    <a href="https://www.linkedin.com/in/beewant/" target="_blank">
        <img src="https://img.shields.io/badge/LinkedIn-0077B5?logo=linkedin&logoColor=white" alt="LinkedIn">
    </a>
</td>

<td>
    <a href="https://beewant.slack.com/" target="_blank">
        <img src="https://img.shields.io/badge/Slack-4A154B?logo=slack&logoColor=white" alt="Slack">
    </a>
</td>

<p>Explore the powerful capabilities of semantic search on the BeeWant platform with this comprehensive tutorial. Learn how to harness the potential of semantic search, whether through providing detailed prompts for text-based searches or utilizing the innovative feature of searching by image. Discover the intricacies of enhancing search accuracy and contextually relevant outcomes by tailoring your queries. Whether you're leveraging the semantic search for text or diving into image-based searches, this tutorial will guide you through the steps to optimize your search experience on the BeeWant platform.</p>

In [ ]:
import json
from IPython.display import HTML, display
import threading
import requests
import base64

# Perform Search by prompt

In [ ]:
# set the base url for the api
url_base = 'https://beewant.com/api/projects/'

# set the api key
beewant_key = ''

# set the headers for the api
headers = {'Authorization': f'Token {beewant_key}','Content-Type': 'application/json'}

### Retrieving results 

In [ ]:
# set seaarch endpoint
url_search = url_base + "semantic-search/"

# set project id
project_id = " "
 
# set payload
payload = json.dumps({
  "project": f"{project_id}",
  "prompt": "Red car",
})
 
# send request 
response = requests.request("POST", url_search, headers=headers, data=payload)

if response.status_code == 200:
    response_initial = response.json()
    print("Success!")
else:
    print("Error: " + str(response.status_code))
    print(response.text)

In [ ]:
# sort the scores
scores_dict = response_initial.get("scores", {})

# Extract relevant information from response_initial
sorted_scores = dict(sorted(scores_dict.items(), key=lambda x: x[1], reverse=True)[:100])

In [ ]:
scores_dict = response_initial.get("scores", {})

# Assuming scores_dict is already sorted, take the top 10
sorted_scores = dict(list(scores_dict.items())[:10])

# Lists to store results
task_similarity_url = []

# List to keep track of threads
threads = []

# Thread function to fetch task info
def fetch_task_info(task, similarity_score):
    url_task_info = f"https://beewant.com/api/tasks/{task}/"
    response = requests.get(url_task_info, headers=headers)
    if response.status_code == 200:
        response_data = response.json()
        first_key = next(iter(response_data["data"]))
        image_url = response_data["data"][first_key]
        task_similarity_url.append((task, similarity_score, image_url))
        # task_similarity.append(similarity_score)
        # first_tasks_url.append(image_url)
    else:
        print(f"Failed to fetch data for task {task}")

In [ ]:
# Create and start threads
for task, similarity_score in sorted_scores.items():
    thread = threading.Thread(target=fetch_task_info, args=(task, similarity_score))
    threads.append(thread)
    thread.start()

# Wait for all threads to complete
for thread in threads:
    thread.join()

In [ ]:
task_similarity_url.sort(key=lambda x: x[1], reverse=True)

In [ ]:
# extract similarities from task_similarity_url
task_similarity = [x[1] for x in task_similarity_url]

# extract urls from task_similarity_url
first_tasks_url = [x[2] for x in task_similarity_url]

### Displaying top images with their infos

In [ ]:
# Create dict called image_info that contains each task id and its similarity score
image_info = {
    task: {"similarity_score": similarity, "task_url": url}
    for task, similarity, url in zip(sorted_scores.keys(), task_similarity, first_tasks_url)
}

In [ ]:
# Create HTML content with clickable images
html_content = "<div style='display: flex; flex-wrap: wrap;'>"

for task_id, info in image_info.items():
    url = info['task_url']
    image_html = f"<a href='{url}' target='_blank'><img src='{url}' style='width: 200px; height: 200px; margin: 10px;' /></a>"
    info_html = f"<div style='margin: 10px;'>ID: {task_id}<br>Similarity score: {info['similarity_score']}</div>"
    
    # Combine image and info HTML
    combined_html = f"<div style='position: relative;'>{image_html}{info_html}</div>"
    
    # Add the combined HTML to the overall content
    html_content += combined_html

html_content += "</div>"

# Display the HTML content
display(HTML(html_content))

# Perform Search by image

In [ ]:
# set the base url for the api
url_search = url_base + "image-search/"

# Path to the image file
image_path = './image.png'

# set project id
project_id = " "

# Read the image and convert it to base64
with open(image_path, 'rb') as image_file:
    base64_image = base64.b64encode(image_file.read()).decode('utf-8')

In [ ]:
params = json.dumps({
    'project': f"{project_id}",
    'image_prompt': f'data:image/jpeg;base64,{base64_image}',
    'bias': 0.0,
    'modelName': 'CLIP',
})

response = requests.post(url_search, headers=headers, data=params)
if response.status_code == 200:
    response_initial = response.json()
    print("Success!")

In [ ]:
scores_dict = response_initial.get("scores", {})

# Extract relevant information from response_initial
sorted_scores = dict(sorted(scores_dict.items(), key=lambda x: x[1], reverse=True)[:10])

In [ ]:
scores_dict = response_initial.get("scores", {})

# Assuming scores_dict is already sorted, take the top 10
sorted_scores = dict(list(scores_dict.items())[:10])

# Lists to store results
task_similarity_url = []

# List to keep track of threads
threads = []

# Thread function to fetch task info
def fetch_task_info(task, similarity_score):
    url_task_info = f"https://beewant.com/api/tasks/{task}/"
    response = requests.get(url_task_info, headers=headers)
    if response.status_code == 200:
        response_data = response.json()
        first_key = next(iter(response_data["data"]))
        image_url = response_data["data"][first_key]
        task_similarity_url.append((task, similarity_score, image_url))
    else:
        print(f"Failed to fetch data for task {task}")
        print(response.text)

In [ ]:
# Create and start threads
for task, similarity_score in sorted_scores.items():
    thread = threading.Thread(target=fetch_task_info, args=(task, similarity_score))
    threads.append(thread)
    thread.start()

# Wait for all threads to complete
for thread in threads:
    thread.join()

In [ ]:
task_similarity_url.sort(key=lambda x: x[1], reverse=True)

In [ ]:
# extract similarities from task_similarity_url
task_similarity = [x[1] for x in task_similarity_url]
# extract urls from task_similarity_url
first_tasks_url = [x[2] for x in task_similarity_url]

### Displaying top images with their infos

In [ ]:
# Create dict called image_info that contains each task id and its similarity score
image_info = {
    task: {"similarity_score": similarity, "task_url": url}
    for task, similarity, url in zip(sorted_scores.keys(), task_similarity, first_tasks_url)
}

In [ ]:
# Create HTML content with clickable images
html_content = "<div style='display: flex; flex-wrap: wrap;'>"

for task_id, info in image_info.items():
    url = info['task_url']
    image_html = f"<a href='{url}' target='_blank'><img src='{url}' style='width: 200px; height: 200px; margin: 10px;' /></a>"
    info_html = f"<div style='margin: 10px;'>ID: {task_id}<br>Similarity score: {info['similarity_score']}</div>"
    
    # Combine image and info HTML
    combined_html = f"<div style='position: relative;'>{image_html}{info_html}</div>"
    
    # Add the combined HTML to the overall content
    html_content += combined_html

html_content += "</div>"

# Display the HTML content
display(HTML(html_content))